# RFM Analizi

In [1]:
# 1. İş Problemi (Business Problem)
# 2. Veriyi Anlama (Data Understanding)
# 3. Veri Hazırlama (Data Preparation)
# 4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
# 5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
# 6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
# 7. Tüm Sürecin Fonksiyonlaştırılması

### 1. İş Problemi (Business Problem)

Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.

In [2]:
# Veri Seti Hikayesi
# https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

# Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının
# 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.

In [3]:
# Değişkenler
#
# InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice: Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.

### 2. Veriyi Anlama (Data Understanding)

In [4]:
import datetime as dt
import pandas as pd

In [5]:
yol = "datasets/online_retail_II.xlsx"

df_ = pd.read_excel(yol, sheet_name="Year 2009-2010")

# df_ diye tanımlayıp daha sonra kopyalamamın sebebi ise verinin okunması zaman aldığı için daha sonrasında df'e dönmek istersem 
# kolaylık olsun diye. Bir daha okutma işlemini yapmamak adına.


df = df_.copy()


In [6]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
df.shape

(525461, 8)

In [8]:
df.isnull().sum()

# Aşağıda görüldüğü üzere "Customer ID" sütununda 107927 adet eksik veri bulunmaktadır.
# Müşteriler üzerinde analiz yapacağımız için ve müşteri no'su eksik olan değerleri direk silmeyi tercih ediyoruz.

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [9]:
# essiz urun sayisi nedir?
df["Description"].nunique()

4681

In [10]:
df["StockCode"].nunique()

4632

In [11]:
# Her bir ürünün faturaya konu olma sayısını bulmak istersek; 
# DİKKAT EDELİM! Bu satılma miktarı değildir !!!

df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: Description, dtype: int64

In [12]:
# Her bir ürünün satılma miktarına bakmak daha sonrasında en çok satılan ürünü bulmak istersek;

df.groupby("Description").agg({"Quantity": "sum"}).head()

# Baktığımızda bir takım satışların eksi olduğunu görüyoruz. Burada bir hata olduğunu anlıyoruz, bunu ileride düzelteceğiz.

,Quantity
Description,
21494,-720
22467,-2
22719,2
DOORMAT UNION JACK GUNS AND ROSES,179
3 STRIPEY MICE FELTCRAFT,690


In [13]:
# Şimdi bu listeyi azalan olarak sıralamak için;

df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [14]:
# Toplam kaç adet fatura kesildiğine bakabilmek için;

df["Invoice"].nunique()

# Totalde 28816 adet fatura kesildiğini gördük.

28816

In [15]:
# Her bir satıştan ne kadarlık bir gelir elde edildiğini bulabilmek için ise;

df["TotalPrice"] = df["Quantity"]*df["Price"]
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom,5.90
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,3.75
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,3.75
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,7.50


In [16]:
# Faturanın dip toplamını görmek istersek (faturanın total tutarını);

df.groupby("Invoice").agg({"TotalPrice": "sum"}).head()

,TotalPrice
Invoice,
489434,505.30
489435,145.80
489436,630.33
489437,310.75
489438,2286.24


### Veri Hazırlama (Data Preparation)

In [17]:
df.shape

(525461, 9)

In [18]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

In [19]:
# Boş olan değerleri çıkaracağız;

df.dropna(inplace=True)

In [20]:
df.describe().T

# Bakıldığında - olan değerler var bunlar iade faturalarından kaynaklanıyor.
# Fatura başında C var ise bunlar iade faturaları.

,count,mean,std,min,25%,50%,75%,max
Quantity,417534.0,12.758815,101.220424,-9360.00,2.00,4.00,12.00,19152.00
Price,417534.0,3.887547,71.131797,0.00,1.25,1.95,3.75,25111.09
Customer ID,417534.0,15360.645478,1680.811316,12346.00,13983.00,15311.00,16799.00,18287.00
TotalPrice,417534.0,19.994081,99.915863,-25111.09,4.25,11.25,19.35,15818.40


In [21]:
# İade Faturaların bulunması;

df[df["Invoice"].str.contains("C", na=False)]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia,-35.40
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia,-9.90
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia,-17.00
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia,-12.60
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia,-35.40
...,...,...,...,...,...,...,...,...,...
524695,C538123,22956,36 FOIL HEART CAKE CASES,-2,2010-12-09 15:41:00,2.10,12605.0,Germany,-4.20
524696,C538124,M,Manual,-4,2010-12-09 15:43:00,0.50,15329.0,United Kingdom,-2.00
524697,C538124,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2010-12-09 15:43:00,2.95,15329.0,United Kingdom,-2.95
524698,C538124,22423,REGENCY CAKESTAND 3 TIER,-1,2010-12-09 15:43:00,12.75,15329.0,United Kingdom,-12.75


In [22]:
# İade faturalarının değili ise;
# Ayrıca bunu tekrardan df'e atayacağız.

df = df[~df["Invoice"].str.contains("C", na=False)]

In [23]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom,5.90
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,3.75
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,3.75
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,7.50


### 4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)

In [24]:
# RFM metriklerini bir hatırlayalım
# Recency, Frequency, Monetary

# Recency analizin yapılıdğı tarih ile satın alımın yapıldığı tarih arasında bir işlemdir.
# Lakin veri setine baktığımızda veri 2009 ila 2011 tarihleri arasında olan bir kısmı kapsamaktadır.
# Yani bizim makul bir tarih bulup ona göre analizimi yürütmemiz gerekir.
# Bu tarih de veri seti üzerindeki en geç tarihin üzerine örneğin 2 gün eklenmesi ile bulunabilir.

df["InvoiceDate"].max()

# Görüldüğü üzere en son tarih 12. ayın 9'unu göstermektedir.

Timestamp('2010-12-09 20:01:00')

In [25]:
today_date=dt.datetime(2010,12,11)

# Görüldüğü üzere today_date değişkeni ile datetime kütüphanesi kullanılarak tanımlama işlemi yapılmıştır.

In [26]:
rfm=df.groupby("Customer ID").agg({ "InvoiceDate": lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                    "Invoice"    : lambda Invoice: Invoice.nunique(),
                                    "TotalPrice" : lambda TotalPrice: TotalPrice.sum()
})

rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93


In [27]:
# Yukarıdaki tablonun sütun isimlerini değiştirmek istersek;

rfm.columns = ['recency', 'frequency', 'monetary']
rfm

,recency,frequency,monetary
Customer ID,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93
...,...,...,...
18283.0,18,6,641.77
18284.0,67,1,461.68
18285.0,296,1,427.00


In [28]:
rfm.describe().T

# Aşağıdaki tablo incelendiğinde;
# recencty'e bakarak %75'lik dilimdeki 135 ile max olan 374 değeri mantıklı gözüküyor.
# frequency'deki %75 olan 5 değeri ile 205 değeri mantıksız değil ama uç değer olduğu da aşikar. 
# Lakin bizim bunları skora dönüştürme işlemimizde yine de sorun çıkarmayacaktır.
# monetary değerindeki min değerin 0 olması istediğimiz ve zaten mantıklı bir şey değildir. Zira alışveriş yapan birisinin;
# 0 TL veya USD tutarında alış veriş yapması mümkün değişdir.

,count,mean,std,min,25%,50%,75%,max
recency,4314.0,91.269819,96.944304,1.0,18.00,53.00,136.0000,374.00
frequency,4314.0,4.454103,8.168658,1.0,1.00,2.00,5.0000,205.00
monetary,4314.0,2047.288659,8912.523243,0.0,307.95,705.55,1722.8025,349164.35


In [29]:
# Yukarıdaki son kısımda bahsedilen durumun üstesinden gelebilmek için;

rfm = rfm[rfm["monetary"]>0]
rfm

,recency,frequency,monetary
Customer ID,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93
...,...,...,...
18283.0,18,6,641.77
18284.0,67,1,461.68
18285.0,296,1,427.00


### RFM Skorlarının Hesaplanması

In [30]:
# Bir önceki kısımda rfm metrikleri hesaplanmıştı, bu bölümde ise rfm skorlarını hesaplayacağız.
# Frequency ve monetary değerleri ne kadar büyük ise bizim içn o kadar iyi, recency ne kadar küçük ise yine bizim için o kadar iyidir.
# Bu sebeple monetary ve frequency'de büyük olanlara yüksek puan; recency'de de düşük olanlara yüksek puan vereceğiz.

rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])

# pd.qcut() metodu ile ilgili değişkeni bölebiliyoruz. Bölerken de kaça bölebileceğimiz ve 
# bunları böldükten sonra nasıl etiketleyebileceğimizi seçebiliyoruz.

rfm

/var/folders/l2/g3cwllyj4_139gx51n27kyn80000gn/T/ipykernel_28473/847875583.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])


,recency,frequency,monetary,recency_score
Customer ID,,,,
12346.0,165,11,372.86,2
12347.0,3,2,1323.32,5
12348.0,74,1,222.16,2
12349.0,43,3,2671.14,3
12351.0,11,1,300.93,5
...,...,...,...,...
18283.0,18,6,641.77,4
18284.0,67,1,461.68,3
18285.0,296,1,427.00,1


In [31]:
# Şimdi ise monetary skorunu hesaplıyoruz;

rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])
rfm

/var/folders/l2/g3cwllyj4_139gx51n27kyn80000gn/T/ipykernel_28473/1363481873.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])


,recency,frequency,monetary,recency_score,monetary_score
Customer ID,,,,,
12346.0,165,11,372.86,2,2
12347.0,3,2,1323.32,5,4
12348.0,74,1,222.16,2,1
12349.0,43,3,2671.14,3,5
12351.0,11,1,300.93,5,2
...,...,...,...,...,...
18283.0,18,6,641.77,4,3
18284.0,67,1,461.68,3,2
18285.0,296,1,427.00,1,2


In [32]:
# Son olarak da frequency değişkeni için skor hesağlıyoruz;

rfm["frequency_score"] = pd.qcut(rfm["frequency"], 5, labels=[1,2,3,4,5])
rfm

ValueError: Bin edges must be unique: array([  1.,   1.,   2.,   3.,   6., 205.]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [33]:
# Yukarıda bir hata alıyoruz, Bu hata, birden çok değer aynı çeyrekliğe karşılık geldiğinde ortaya çıkar. 
# Çünkü algoritma ortak sayıyı hangi kategoriye koyacağına karar veremez.
# Anlatması biraz uzun sürüyor lakin hatanın detaylarına;
# "https://medium.datadriveninvestor.com/valueerror-bin-edges-must-be-unique-71512ff2257d" linkinden ulaşılabilir.


# Hatanın çözümü için ise;

rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm

# eklemesini yapmak gerekiyor.

/var/folders/l2/g3cwllyj4_139gx51n27kyn80000gn/T/ipykernel_28473/2496862403.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])


,recency,frequency,monetary,recency_score,monetary_score,frequency_score
Customer ID,,,,,,
12346.0,165,11,372.86,2,2,5
12347.0,3,2,1323.32,5,4,2
12348.0,74,1,222.16,2,1,1
12349.0,43,3,2671.14,3,5,3
12351.0,11,1,300.93,5,2,1
...,...,...,...,...,...,...
18283.0,18,6,641.77,4,3,5
18284.0,67,1,461.68,3,2,2
18285.0,296,1,427.00,1,2,2


In [34]:
# RFM skorunu oluşturmak için ise recency_score ile frequency_score'u yan yana getireceğiz.
# Monetary skorunu gözlemlemek adına çağırdık.

rfm["RFM_SCORE"] = ( rfm["recency_score"].astype(str) +
                     rfm["frequency_score"].astype(str))
rfm

/var/folders/l2/g3cwllyj4_139gx51n27kyn80000gn/T/ipykernel_28473/3917126592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["RFM_SCORE"] = ( rfm["recency_score"].astype(str) +


,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE
Customer ID,,,,,,,
12346.0,165,11,372.86,2,2,5,25
12347.0,3,2,1323.32,5,4,2,52
12348.0,74,1,222.16,2,1,1,21
12349.0,43,3,2671.14,3,5,3,33
12351.0,11,1,300.93,5,2,1,51
...,...,...,...,...,...,...,...
18283.0,18,6,641.77,4,3,5,45
18284.0,67,1,461.68,3,2,2,32
18285.0,296,1,427.00,1,2,2,12


In [35]:
rfm.describe().T

# Bakılıdında "RFM_SCORE" tip olarak numerik bir değer olmadığı string olduğu için burada analizi çıkmadı.

,count,mean,std,min,25%,50%,75%,max
recency,4312.0,91.172542,96.861457,1.00,18.0000,53.00,136.0000,374.00
frequency,4312.0,4.455705,8.170213,1.00,1.0000,2.00,5.0000,205.00
monetary,4312.0,2048.238236,8914.481280,2.95,307.9875,706.02,1723.1425,349164.35


In [36]:
# Bizler kodu 55 olan şampiyonlarımızı görmek istersek;

rfm[rfm["RFM_SCORE"] == "55" ]

# 457 adet şampiyon müşterimizin olduğunu görüyoruz.

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE
Customer ID,,,,,,,
12415.0,11,7,19543.84,5,5,5,55
12431.0,9,13,4370.52,5,5,5,55
12471.0,10,49,20139.74,5,5,5,55
12472.0,5,13,11308.48,5,5,5,55
12474.0,14,13,5048.66,5,5,5,55
...,...,...,...,...,...,...,...
18225.0,1,15,7545.14,5,5,5,55
18226.0,14,15,6650.83,5,5,5,55
18229.0,2,10,3526.81,5,5,5,55


### RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi

In [37]:
# Regex nedir araştırmasını yap!!!

seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map, regex=True )
rfm

/var/folders/l2/g3cwllyj4_139gx51n27kyn80000gn/T/ipykernel_28473/1150339466.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map, regex=True )


,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.0,165,11,372.86,2,2,5,25,cant_loose
12347.0,3,2,1323.32,5,4,2,52,potential_loyalists
12348.0,74,1,222.16,2,1,1,21,hibernating
12349.0,43,3,2671.14,3,5,3,33,need_attention
12351.0,11,1,300.93,5,2,1,51,new_customers
...,...,...,...,...,...,...,...,...
18283.0,18,6,641.77,4,3,5,45,loyal_customers
18284.0,67,1,461.68,3,2,2,32,about_to_sleep
18285.0,296,1,427.00,1,2,2,12,hibernating


In [38]:
rfm[["segment", "recency", "monetary", "frequency"]].groupby("segment").agg(["mean", "count"])

# Aşağıya baktığımızda;
# at_risk'de 611 kişi olduğunu, 
# ortalama olarak 152 gündür işlem yapmadıklarını ve 
# yine ortalama olarak 1188 TL veya USD'lik harcama yaptıklarını
# ve 3 kere alış veriş yaptıklarını görebiliriz.

# Yukarıdaki gibi yorumlarda bulunabiliriz.

recency           monetary        frequency      
                           mean count         mean count       mean count
segment                                                                  
about_to_sleep        53.819242   343   441.320000   343   1.201166   343
at_risk              152.158756   611  1188.878316   611   3.073650   611
cant_loose           124.116883    77  4099.450000    77   9.116883    77
champions              7.119155   663  6852.264167   663  12.553544   663
hibernating          213.885714  1015   403.977836  1015   1.126108  1015
loyal_customers       36.287062   742  2746.067353   742   6.830189   742
need_attention        53.265700   207  1060.357005   207   2.449275   207
new_customers          8.580000    50   386.199200    50   1.000000    50
potential_loyalists   18.793037   517   729.510986   517   2.017408   517
promising             25.747126    87   367.086782    87   1.000000    87

In [39]:
# Diyelim ki, belirli bir gruba yönelik bir çalışma yapılacak ve bu grubun ID'leri isteniyor;

rfm[rfm["segment"] == "need_attention"]

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12349.0,43,3,2671.14,3,5,3,33,need_attention
12369.0,49,3,1791.15,3,4,3,33,need_attention
12371.0,45,3,2179.42,3,5,3,33,need_attention
12374.0,57,3,2246.29,3,5,3,33,need_attention
12389.0,38,3,1433.33,3,4,3,33,need_attention
...,...,...,...,...,...,...,...,...
18078.0,61,2,508.47,3,3,3,33,need_attention
18112.0,54,2,423.27,3,2,3,33,need_attention
18136.0,45,2,847.03,3,3,3,33,need_attention


In [40]:
rfm[rfm["segment"] == "need_attention"].index

Float64Index([12349.0, 12369.0, 12371.0, 12374.0, 12389.0, 12412.0, 12418.0,
              12644.0, 12656.0, 12909.0,
              ...
              17741.0, 17823.0, 17834.0, 18016.0, 18060.0, 18078.0, 18112.0,
              18136.0, 18141.0, 18244.0],
             dtype='float64', name='Customer ID', length=207)

In [41]:
new_df=pd.DataFrame()
new_df["new_cunstomer_id"] = rfm[rfm["segment"] == "need_attention"].index
new_df

,new_cunstomer_id
0,12349.0
1,12369.0
2,12371.0
3,12374.0
4,12389.0
...,...
202,18078.0
203,18112.0
204,18136.0
205,18141.0


In [42]:
# id'lerin sonundaki sıfırlar olmasın istersek;

new_df["new_cunstomer_id"] = new_df["new_cunstomer_id"].astype(int)
new_df

,new_cunstomer_id
0,12349
1,12369
2,12371
3,12374
4,12389
...,...
202,18078
203,18112
204,18136
205,18141
